# Download WEB of Science

In [1]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from pathlib import Path
import time
import glob
import os
import sys
import getpass

In [2]:
downloads='Downloads'
path='{}/{}'.format(os.getenv("HOME"),downloads)

In [34]:
# Download file with firefox bypassing popup : Selenium Python
# See: https://stackoverflow.com/q/52907319/2268280
# TODO: NOT WORKING YET!
fp = webdriver.FirefoxProfile()
fp.set_preference("browser.download.folderList", 2)
fp.set_preference("browser.download.manager.showWhenStarting", False)
fp.set_preference("browser.download.dir", path)
#See: https://www.toolsqa.com/selenium-webdriver/how-to-download-files-using-selenium/
#   and https://www.sitepoint.com/mime-types-complete-list/
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain")

In [4]:
home = str(Path.home())
try:
    browser = webdriver.Firefox(executable_path=
          '{}/.wdm/geckodriver/v0.23.0/linux64/geckodriver'.format(os.getenv("HOME")),
                               firefox_profile=fp)
except:
    browser = webdriver.Firefox(
            executable_path= GeckoDriverManager().install(),
            firefox_profile=fp)

In [5]:
url="https://login.webofknowledge.com"
browser.get(url)

In [ ]:
email=input('email:')

In [7]:
passwd=getpass.getpass('password:')

password: ········


In [8]:
browser.find_element_by_name("username").send_keys("{}".format(email))

In [9]:
browser.find_element_by_name("password").send_keys("{}".format(passwd))

In [10]:
button=browser.find_elements_by_tag_name('button')

In [11]:
button[0].click()

In [12]:
try:
    browser.find_element_by_link_text('continue and establish a new session').click()
except:
    pass

In [13]:
time.sleep(10)
qq=browser.find_element_by_name("currUrl")

In [14]:
sid=qq.get_attribute('value').split('SID=')

In [15]:
sid[-1]

'8Cs8q27XZNuZ9OVr6N8'

In [16]:
urls="https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID={}&product=WOS&search_mode=AdvancedSearch".format(sid[-1])

In [17]:
browser.get(urls)

In [18]:
browser.find_element_by_name("value(input1)").send_keys("CU=Colombia")

In [19]:
kk=browser.find_element_by_id('search-button').click()

In [20]:
urlr=browser.find_element_by_id('hitCount').get_attribute('href')

In [21]:
browser.get(urlr)

## Loop for each 500 entries

In [22]:
try:
    results=eval(browser.find_element_by_id('hitCount.top'
                               ).text.replace(',',''))
except:
    results=0
if not results:
    sys.exit()

In [23]:
results

104477

In [24]:
wostep=500
lstq=results%wostep
results=results-lstq

In [25]:
steps=list(range(1,results+wostep,wostep))

In [26]:
n=0
try:
    ll=[ n.split('(')[-1].split(')')[0] for n in sorted(glob.glob("{}/savedrecs(*.txt".format(path))) ]
    ll=sorted([eval(l) for l in ll])[::-1]
    n=ll[0]
except:
    pass

In [27]:
n

0

In [28]:
#steps=steps[n+1:]
steps[0]

1

In [ ]:
slp=0.5
nextlooptime=10
markFrom=1
martTo=500
nstep=499
for step in steps:
    markFrom=step
    if step==steps[-2]:
        nstep=lstq-1
    martkTo=step+nstep
    print(markFrom,martkTo)
    
    #if step>1500:
    #    break
#if True:
    try:
        kk=browser.find_element_by_name('export').click()
        kk=browser.find_element_by_name(
            'Export to Other File Formats').click()
    except:
        kk=browser.find_elements_by_tag_name("button")
        i=[w.text for w in kk].index('Export to Other File Formats')
        kk[i].click()

    time.sleep(slp)
    kk=browser.find_element_by_id('numberOfRecordsRange').click()

    time.sleep(slp)
    kk=browser.find_element_by_id('markFrom').clear()
    kk=browser.find_element_by_id('markFrom').send_keys("{}".format(markFrom))

    time.sleep(slp)
    kk=browser.find_element_by_id('markTo').clear()
    kk=browser.find_element_by_id('markTo').send_keys("{}".format(martkTo))

    time.sleep(slp)
    #Take the id of the <span id='select2-bib_fields-container'
    kk=browser.find_element_by_id('select2-bib_fields-container')
    #make click to activate the hidden `<li` options!
    kk.click()

    #Get the dinamically generated new `<li` options
    i=[w.text for w in browser.find_elements_by_tag_name('li')].index('Full Record and Cited References')

    time.sleep(slp)
    browser.find_elements_by_tag_name('li')[i].click()

    #The same for the next dinamycal menu

    time.sleep(slp)
    kk=browser.find_element_by_id('select2-saveOptions-container').click()

    i=[w.text for w in browser.find_elements_by_tag_name('li')].index('Plain Text')

    time.sleep(slp)
    browser.find_elements_by_tag_name('li')[i].click()

    time.sleep(slp)
    #Firefox download popup must be configured to save automatically the first time!!!!!
    browser.find_element_by_id("exportButton").click()

    wtr=range(10)
    for wait in wtr:
        if glob.glob("{}/savedrecs({}).txt".format(path,n+1)):
            n=n+1
            break
        else:
            print('.',end='')
            time.sleep(nextlooptime)
    print('')
    #TODO: break for n=0
    if wait==wtr[-1] and n>0:
        print('WARNING: Download failed')

    time.sleep(slp)
    kk=browser.find_element_by_class_name("quickoutput-overlay-buttonset")
    kk.find_element_by_tag_name('a').click()
    time.sleep(nextlooptime)

#TODO:
# * Make a logout